## Deriving xpriv Fom xpub and a Child Private Key

This notebook contains executable cells that illustrate the points made in [Deterministic Wallets, Their Advantages and Understated Flaws](https://bitcoinmagazine.com/technical/deterministic-wallets-advantages-flaw-1385450276)

The code used to illustrate the points in the article was taken from the [patch-1 branch of pybitcointools](https://github.com/vbuterin/pybitcointools/tree/patch-1/cryptos), which was also written by the author of the article (and creator of Ethereum Vitalik Buterin). According to the readme in the main branch, this library is no longer supported, but has been forked into another library linked to in the readme. However, the code below does not work as expected when using this new library, so I've extracted into this repository the subset of code from pybitcointools to provided the example from the article.


In [2]:
# Importing from the excerpted pybitcointools code
from pybctexcerpt import *

### Create a Master Private Key and Derive the Public Key

In [3]:
# Using the same seed as the article
w = bip32_master_key('qweqweqweqweqwe')

In [4]:
print(w)

xprv9s21ZrQH143K2KhRQVuMqhz798mvW89J1aJMWEKNfZzv3BPAgBc4TH59K8ZcLt8RrNJvbUzA72A92Grm3MorG2FnFaoZ7B8SDTYXgUBALoi


In [9]:
wp = bip32_privtopub(w)
print(wp)

xpub661MyMwAqRbcEomtWXSNCqvqhAcQuas9NoDxJcizDuXtuyiKDivK15PdAPVkPwVXT9rFbjAnE9P3sLh6xnDawXF1uUXrruH1UvALHF89qdP


### Derive a Child Key, and Use it With Master xpub to Recover Master xpriv

In [19]:
w0 = bip32_ckd(w,0)
print(w0)

xprv9uyTuGongdyZAMxZ2euUBbpsAdtE2nxFBmcQn89UT4ZyzrMg5TXD7azCnsnpH9Q7yrYgG7nVakE6BTxJUarLrDA28VxS3ZWDsgYWZUxtNiH


In [20]:
cracked = crack_bip32_privkey(wp,w0)
print(cracked)

xprv9s21ZrQH143K2KhRQVuMqhz798mvW89J1aJMWEKNfZzv3BPAgBc4TH59K8ZcLt8RrNJvbUzA72A92Grm3MorG2FnFaoZ7B8SDTYXgUBALoi


In [21]:
cracked == w

True

## How far back up the tree can you go?

In [5]:
# Parent public and private keys
w = bip32_master_key('not a good seed')
wp = bip32_privtopub(w)

print(f"w is {w}")
print(f"wp is {wp}")

w is xprv9s21ZrQH143K2hvSbgbZDJCm7iz4WB7UeM4p4rpcwDec2rHZD28k5koHhdkodpSLWW9VirG4DfxwC63UMkP5rYdvxKV8ozrkY7Ad4kxBZTT
wp is xpub661MyMwAqRbcFBzuhi8ZaS9VfkpYudqL1ZzQsFEEVZBauechkZSzdZ7mYvff4xreYraAQ6ooyByGjsoC3FeRNZPQgaWhXPqLsjjRy2aE2A4


In [16]:
# Derive some child keys
w0 = bip32_ckd(w,0)
wp0 = bip32_privtopub(w0)
print(w0)
print(wp0)

w00 = bip32_ckd(w0,0)
wp00 = bip32_privtopub(w00)
print(w00)
print(wp00)

w000 = bip32_ckd(w00,0)
wp000 = bip32_privtopub(w000)
print(w000)
print(wp000)

xprv9vXzy8F2R3nCJQmsDZJgFKJvVSRdjwpNADUigaWnQf6ETnWYg6Q9bg8fYd9rC4KyLwRCXbQSRuTgUwuS62AnK87SErvPz6bjP3Y7Z5s8AW1
xpub69XMNdmvFRLVWtrLKaqgcTFf3UG89QYDXSQKUxvPxzdDLaqhDdiQ9UT9Pu9k6SLdetHpFB3GeAvqGrMDGUQf4SrRvXMkde3yZ5xVkgkjtg7
xprv9wCGDa5GiRzmL2Qp9K2yjS4fjLz9YkiQKQcWt79Swq3r2QRppfWGexSxk6txQJy3e2YD9WMXxbLsPA7wyx1uqUS1UCqNLqFJ8r8Ymtiv4La
xpub6ABcd5cAYoZ4YWVHFLZz6a1QHNpdxDSFgdY7gVZ4WAapuCkyNCpXCkmSbN8Ap39MdJBF48R8U8SpnK9Xt5hzoc1nqDviVxeqYFqru2tghU9
xprv9yty3GoxwpYuqBu2tNsJ8ueVaZWkhvmGLPxiKST6jwmhECUoCdrdb5AbUjXPJXC8MD1NUBouz1ae9grUZAGGJW3avua1SAZFfsQg7JznGuo
xpub6CtKSnLrnC7D3fyVzQQJW3bE8bMF7PV7hctK7priJHJg6zowkBAt8sV5L2Azk9bkz7g6q4uKvw4VMDbGXRReiFHtW9na5MbrGGKuDGB9SMU


In [7]:
# What do we get if we crack child key w000 using wp?
cracked000 = crack_bip32_privkey(wp,w000)
print(cracked000)

xprv9s21ZrQH143K2hvSbgbZDJCm7iz4WB7UeM4p4rpcwDec2rHZD28k5koHhe9aTSpQhTsTVQrxGMRHsZrfh7V5ctn3D2eCqfbCLCsHCeCNE9k


In [11]:
# Did we get a private key in the hierarchy?
if cracked000 == w0:
        print('crack of w000 gave us w0')
elif cracked000 == w00:
        print('crack of w000 gave us w00')
elif cracked000 == w000:
        print('crack of w000 gave us w000')
else:
        print('crack of w000 yielded unrecognized key')
            

crack of w000 yielded unrecognized key


In [21]:
# But... since we know w000 and can derive wp00 from the xpub (wp)

cracked000 = crack_bip32_privkey(wp00,w000)
print(cracked000)
cracked000 == w00

xprv9wCGDa5GiRzmL2Qp9K2yjS4fjLz9YkiQKQcWt79Swq3r2QRppfWGexSxk6txQJy3e2YD9WMXxbLsPA7wyx1uqUS1UCqNLqFJ8r8Ymtiv4La


True

In [22]:
# And if we crack again...
cracked00 = crack_bip32_privkey(wp0,cracked000)
print(cracked00)
cracked00 == w0

xprv9vXzy8F2R3nCJQmsDZJgFKJvVSRdjwpNADUigaWnQf6ETnWYg6Q9bg8fYd9rC4KyLwRCXbQSRuTgUwuS62AnK87SErvPz6bjP3Y7Z5s8AW1


True